## Introduction

A weather forecasting model might use yesterday's temperature to predict tommorow's temperature.

But could it predict the temperature in New York City if it was trained solely on the daily temperature of Denver? What if it was trained on the temperature of 10 different cities spread across the world? What if these cities were within 50 miles of New York City?

In simple situations, it's useful to train an LSTM (Long Short-Term Memory) model on a single time series. By learning what happened in the past, the LSTM model predicts what will happen in the future for a specific situation.

This begs the question: How "specific" does the situation have to be? In some instances, "situation" may refer to the physical location of a signal.

Let's take a more tangible example: Let's say I dropped a couple of rocks into a lake and they ripple waves out in different directions. Could I use machine learning to predict the height of the water over time at one location, based on the height of the water at other locations?

That's the kind of question I'm trying to answer with this project.

## Training an LSTM model on different locations of a 2D wave

Previously, I have simulated a 2 dimensional wave interference pattern and saved the raw data in the form of a PyTorch tensor. The raw data includes the signal's value over time at all locations in a limited spacial grid.

In this notebook, I train an LSTM model on multiple coordinates of the wave simulation, and then test its prediction against the signal at other coordinates.

The LSTM model in question consists of one LSTM layer. Multiple hyperparameters may be adjusted for the model, affecting its results in interesting ways:
* signal input size
* learning rate
* number of layers


This project is broken down into the following steps (which will be explained in more detail later):

* Split data into training and test data. This done by categorizing each of the signal coordinates as "test or "train".
* Prepare data for proper batch training using a "rolling window" method. This splits data into inputs and labels.
* Create LSTM Neural Network Model and train the model with training signal data.
* Test the Model on multiple test signals.


# Import Libraries

In [1]:
#Import libraries
#Import wavetorch module
from src import wavetorch

#import python libraries
import torch
import numpy as np
import torch.optim as optim
import json
import random

#Visualization libraries
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import colors

#Write Libraries
from pathlib import Path
import pandas as pd

c:\Users\adamq\anaconda3\envs\wavetorch\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# Configure GPU/CPU Usage

In [2]:
#Configure GPU/CPU Usage for Tensor operations

#Use GPU if CUDA compatable GPU is available
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  
print(dev)

#Set default tensors to be type float
torch.set_default_tensor_type('torch.cuda.FloatTensor')

cuda:0


In [3]:
"""
dev="cpu"
device = torch.device(dev)  
print(dev)
torch.set_default_tensor_type('torch.FloatTensor')
"""

'\ndev="cpu"\ndevice = torch.device(dev)  \nprint(dev)\ntorch.set_default_tensor_type(\'torch.FloatTensor\')\n'

# Open Raw Data

The raw data is stored in the form of a 3 dimensional Torch tensor. The first dimension represents time. The second and the dimensions represent the spacial coordinates x, y respectively.

Furthermore, stored metadata is opened, representing time step and spacial step sizes.

In [4]:
#Configure open paths
open_folder = "exports"
open_path = "{}\\{}\\".format(str(Path.cwd()), open_folder)

In [5]:
#Open wave simulation metadata
with open(open_path+'data1 metadata.json') as f:
    u_metadata=json.load(f)
u_metadata

{'dx': 0.5, 'dy': 0.5, 'dt': 0.006666666666666667, 'c': 30, 'N_t': 2400}

In [6]:
#Open numpy array to torch tensor
u_tensor = torch.load(open_path + 'data1.pt')
u_tensor

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0

In [7]:
#Get dimensions
N_t = u_tensor.shape[0]
N_x = u_tensor.shape[1]
N_y = u_tensor.shape[2]
t_array = np.linspace(0, N_t*u_metadata['dt'], N_t)
u_tensor.shape

torch.Size([2402, 480, 640])

# Format Raw Data and split into training/test data
Every possible coordinate (i.e. the signal at each location) is defined from the raw data tensor.

After every coordinate is collected into a list, a random assortment of coordinates are taken and put into a Pandas DataFrame, where they are labeled as either "test" or "train" data.

In [8]:
#Get a list of all possible coordinates in u_tensor
#Initialize list
u_tensor_coordinates=[]
#Loop through the x and y axes of u_tensor
for i in range(0, N_x):
    for j in range(0, N_y):
        u_tensor_coordinates += [(i,j)]

#Take random coordinates
u_tensor_coordinates = random.choices(u_tensor_coordinates, k=20)
u_tensor_coordinates

[(370, 242),
 (26, 351),
 (277, 496),
 (390, 353),
 (117, 228),
 (54, 404),
 (347, 126),
 (137, 175),
 (354, 61),
 (95, 286),
 (173, 153),
 (144, 2),
 (357, 515),
 (140, 561),
 (271, 220),
 (334, 130),
 (169, 273),
 (337, 579),
 (17, 210),
 (479, 156)]

In [9]:
#Generate dataframe containing coordinates
df_coord = wavetorch.generate_labels(u_tensor=u_tensor,
                                      source_coordinates=[],
                                      loc_coordinates=u_tensor_coordinates)
df_coord

""
"(370, 242)"
"(26, 351)"
"(277, 496)"
"(390, 353)"
"(117, 228)"
"(54, 404)"
"(347, 126)"
"(137, 175)"
"(354, 61)"
"(95, 286)"


In [10]:
#Split into test and validation sets
df_coord['Set'] = np.nan
df_coord.loc[df_coord.index[0:5],'Set'] = 'train'
df_coord.loc[df_coord.index[15::],'Set']  = 'test'
df_coord[0:5]

,Set
"(370, 242)",train
"(26, 351)",train
"(277, 496)",train
"(390, 353)",train
"(117, 228)",train


# Define Function for Formatting Training Data

A one dimensional signal needs to be properly formatted into an input and an output i.e. "label" for an LSTM model to train on it.

The function create_dataset generates multiple inputs and outputs per signal through multiple overlapping windows. Roughly speaking, each window has labels indicating what happens in the signal right after the input.

With a defined window length "n_window", create_dataset iterates through different values of n to extract the input signal between values "n" and "n + n_window". This is done until the prediction label reaches the end of the signal.

For each point within a window, an output signal/label of length "n_predict" is generated. Thus, the input data is 2 dimensional and the output data is 3 dimensional. 

In [11]:
#split into multiple signals using a rolling window
def create_dataset(data, n_window, n_predict):
    #initialize data lists
    data_x = []
    data_y = []
    #initialize data
    for n in range(0, len(data)-n_window-n_predict):
        #Get training data
        x = data[n:n+n_window]
        y = []
        #Define y label of length n_predict based on x
        for m in range(0, n_window):
            y += [data[n+m+1:n+m+1+n_predict]]
        #append training data and label to final format
        data_x += [x]
        data_y += [y]
    data_x = torch.Tensor(data_x).detach().unsqueeze(dim=2)
    data_y = torch.Tensor(np.array(data_y)).detach()
    return data_x, data_y

# Define LSTM Model

In PyTorch, neural networks are objects and can be defined as classes. The LSTM class is created with the attributes defined as class arguments:

* Input size is the size of the signal to be trained on
* Hidden size is the size of the hidden layer in the LSTM
* Output size is the size of the predicted signal

Furthermore, the activation function is defined (in this case a linear function).



In [12]:
#Create neural network class
class ModelLSTM(torch.nn.Module):
    
    #create neural network
    def __init__(self, input_size, hidden_size, output_size):
        super(ModelLSTM, self).__init__()
        
        #set parameters
        #batch size ie signal size
        self.input_size = input_size
        #hidden layer size
        self.hidden_size = hidden_size
        #output size
        self.output_size = output_size
        
        #LSTM layer 1
        self.lstm = torch.nn.LSTM(input_size=input_size,
            hidden_size=hidden_size,
            num_layers=3,
            batch_first=True, dropout=0.4)
        self.linear = torch.nn.Linear(in_features=hidden_size, out_features=output_size)
        
    #activation
    def forward(self, x):
        lstm_out, _ = self.lstm(input=x)
        # extract only the last time step
        out = self.linear(lstm_out)
        return out  

# Define Training Process

Once an LSTM model is created, it needs to be trained systematically on the data. This is the purpose of the train_LSTM function.

Before training is done, there are some initial steps in the function:

* Invoke the previous create_dataset() function to generate training inputs ("X_train") and training labels ("y_train") from the raw data.
* Format the training data into a PyTorch DataLoader object. The main thing to note about the DataLoader is that it shuffles the training data with the argument shuffle=True.
* Define the loss function and the optimizer that will train the model according to the loss .

Then, the LSTM model is trained on the DataLoader through a predetermined number of epochs. After the loss is initialized (for training performance tracking), the training process iterates through each batch in the DataLoader with the following steps:

* Set the gradient to 0.
* Make a prediction. (How will the training time series look next?)
* Compute the loss between the prediction "y_pred" and the DataLoader's actual training label "y_batch".
* Append the batch's loss the the epoch's running loss.
* Backpropagate the neural network using the loss



In [19]:
#Function for training neural network

def train_LSTM(LSTM, data, n_window, n_predict, batch_size, learning_rate, momentum, n_epoch, coord, sample, list_loss, n_skip):
    
    #initialize data train as input
    X_train, y_train = create_dataset(data=data, n_window=n_window, n_predict=n_predict)
    #initialize torch's dataloader module to format training data
    loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size,
                                         generator=torch.Generator(device='cuda'))
    #initialize loss function
    criterion = torch.nn.MSELoss()
    #initialize learning method
    optimizer = optim.SGD(LSTM.parameters(), lr=learning_rate, momentum=momentum)

    #train entire batch of data n_epoch times
    for n in range (0, n_epoch):
        
        #Initialize loss for the epoch
        running_loss = 0.0
        batch_count = 0
        #iterate through each windowed signal and i
        #ts label
        for X_batch, y_batch in loader:

            #Clear cache memory between each batch
            torch.cuda.empty_cache()

            LSTM.train()
            #set gradient
            optimizer.zero_grad()

            #get prediction x_train
            y_pred = LSTM(X_batch)
            #get loss function calculation (residual)
            loss = criterion(y_pred, y_batch)
            #append loss to loss array/list
            running_loss += loss.item()
            #backpropagate
            loss.backward()
            optimizer.step()

            #increase batch_count by 1
            batch_count += 1

            #delete objects out of memory
            del y_pred
        
        list_loss += [running_loss/batch_count]

        pred_train = LSTM(X_train)
        #plot result against the original time series
        plt.figure()
        plt.plot(t_array[n_skip::], data, label='actual data')
        plt.plot(t_array[n_window+n_skip+n_predict:n_window+n_skip+n_predict+pred_train.shape[0]], pred_train[:, -1, -1].cpu().detach().numpy(), 'g-', label='predictions')
        plt.title('LSTM with window length {}; Learning Rate {}; Sample {}; Coordinate {}; Epoch {}; Epoch Count {}'.format(str(n_window), str(learning_rate), str(sample), str(coord),str(n), str(n_epoch)),
                                                                                                                            wrap=True)
        plt.legend()
        filename='train_window{}lr{}sample{}coord{}epoch{}epoch_count{}.png'.format(str(n_window), str(learning_rate), str(sample), str(coord),str(n),
                                                                                  str(n_epoch))
        save_path="{}\\exports\\plots\\train_plots\\{}".format(str(Path.cwd()), filename)
        plt.savefig(save_path)

    
        
    
    return LSTM, list_loss

# Create LSTMs and Train them

The training processes are now applied with set hyperparameters, such as learning rate, number of epochs, etc.

Several things to note:
* I opt for a specific prediction length "n_predict". This well determine how many time series into the future the time series predicts.
* I decide to iterate through a list of different window lengths for training to see the difference in effects.

After an LSTM is created and trained with the previously defined functions, they are appended to a list for testing.

In [50]:
#initialize lists of hyperparameters
#window size list
list_n_window = [200, 800]

In [51]:
#initialize list to get models
list_LSTM = []
#Set hyperparameters
learning_rate = 0.05
n_epoch=15
n_predict=200
hidden_size=50
batch_size=1
#Skip the first n_skip # of samples because often the signal is at 0 for the first few samples
n_skip = 500


#loop through hyperparameters
for n_window in list_n_window:
    
    #initialize neural network
    LSTM = ModelLSTM(input_size=1, hidden_size=hidden_size, output_size=n_predict)

    #initialize list of loss values (for plotting over time)
    list_loss = []

    #Message
    print('Training LSTM with window length {}'.format(str(n_window)))
    #loop through each training data point
    for n in range(0, len(df_coord[df_coord['Set']=='train'].index)):
        
        list_loss_coord = []

        print('training coordinate {} out of {}'.format(str(n), str(len(df_coord[df_coord['Set']=='train'].index))))
        #Extract coordinate of data from index
        coord = df_coord[df_coord['Set']=='train'].index[n]
        #Use coordinate to get data
        ts = u_tensor[:,coord[0], coord[1]]
        #Skip the first n_skip # of smaples
        ts = ts[n_skip::]
        #convert from torch tensor to numpy array
        ts = ts.cpu().numpy()
        #Set LSTM
        LSTM, list_loss_coord = train_LSTM(LSTM=LSTM, data=ts, n_window=n_window, n_predict=n_predict, batch_size=batch_size, learning_rate=learning_rate, momentum=0.9, n_epoch=n_epoch, coord=coord, sample=n,
                          list_loss=list_loss_coord, n_skip=n_skip)
        
        list_loss += list_loss_coord

        #Plot loss just for the coordinate
        plt.figure()
        plt.plot(list_loss_coord)
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.title('Average Loss per Epoch for Coordinate {} (Sample {}): Window length {}; Learning Rate {}; Epoch Count {}'.format(str(coord), str(n), str(n_window), str(learning_rate), str(n_epoch)),
                  wrap=True)
        filename='training_loss_sample{}_coord{}_window{}lr{}n_epoch{}.png'.format(str(n), str(coord), str(n_window), str(learning_rate), str(n_epoch))
        save_path="{}\\exports\\plots\\training_loss_plots\\{}".format(str(Path.cwd()), filename)
        plt.savefig(save_path)

    #Plot loss over time
    plt.figure()
    plt.plot(list_loss)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.title('Average Loss per Epoch: Window length {}; Learning Rate {}; {} Epochs per Coordinate'.format(str(n_window), str(learning_rate), str(n_epoch)), wrap=True)
    filename='training_loss_window{}lr{}n_epoch{}.png'.format(str(n_window), str(learning_rate), str(n_epoch))
    save_path="{}\\exports\\plots\\training_loss_plots\\{}".format(str(Path.cwd()), filename)
    plt.savefig(save_path)

    #append LSTM to list
    list_LSTM += [LSTM]

Training LSTM with window length 200
training coordinate 0 out of 5
training coordinate 1 out of 5
training coordinate 2 out of 5
training coordinate 3 out of 5
training coordinate 4 out of 5


# Test LSTM on Test Set

Recall that previously a hold-out "test set" of coordinates for the signal was defined. These are locations of the wave simulation that an LSTM is not trained on.

This is essentially like seeing if we can predict the temperature of Denver after developing algorithms off of New York and Los Angeles's weather.

For each testing process (for a particular LSTM, set of hyperparameters, and location) the following happens:

* The signal is extracted from the wave simulation and the specified coordinate
* Several signal components are used:
    * The actual signal for test comparison. This includes the first portion used to predict the signal in "future time", as well as how the signal actually looks in "future time".
    * Just the first portion used to predict the signal, ie the test input.
* Format the test input for prediction.
* Get the prediction off of the input using the LSTM model.
* Plot the actual signal and the prediction for comparison.



In [52]:
#Skip the first n_skip # of samples because often the signal is at 0 for the first few samples
n_skip = 500

#Iterate through each LSTM
for m in range(0, len(list_n_window)):
    #Get window length
    n_window = list_n_window[m]
    #Get LSTM
    LSTM = list_LSTM[m]
    #Iterate through each test coordinate
    for n in range(0, len(df_coord[df_coord['Set']=='test'].index)):
        LSTM.eval()
        print('Window length {}; Predicting test signal {} out of {}'.format(str(n_window), str(n), str(len(df_coord[df_coord['Set']=='test'].index))))
        #Get coordinate
        coord = df_coord[df_coord['Set']=='test'].index[n]
        #Extract signal from torch tensor at coordinate
        ts = u_tensor[:,coord[0], coord[1]]
        #convert from torch tensor to numpy array
        ts = ts.cpu().numpy()

        #Get the actual signal from the start (after skipping first n_skip # of samples)
        ts_actual = ts[n_skip::]
        #Get the initial signal from which to extrapolate predictions
        ts_test = ts[n_skip:n_window+n_skip+1]
        
        #Get test data
        X_test = torch.Tensor(ts_test).detach().unsqueeze(dim=1).unsqueeze(dim=0)
        #Get prediction
        y_pred = LSTM(X_test)[:,-1,:]

        #plot prediction
        plt.figure()
        plt.plot(t_array[n_skip:len(ts_actual)+n_skip], ts_actual, label='actual data')
        plt.plot(t_array[n_window+n_skip+1:n_window+n_skip+1+len(y_pred[0])], y_pred[0].cpu().detach().numpy(), label='predicted data')
        plt.title('LSTM window length {} Prediction; Learning Rate {}; Test sample {}; Coordinate {}; {} Epochs per Coordinate'.format(str(n_window), str(learning_rate), str(n), str(coord),
                                                                                                                                       str(n_epoch)), wrap=True)
        plt.legend()
        filename='pred_window{}lr{}sample{}coord{}n_epoch{}.png'.format(str(n_window), str(learning_rate), str(n), str(coord), str(n_epoch))
        save_path2="{}\\exports\\plots\\test_plots\\{}".format(str(Path.cwd()), filename)
        plt.savefig(save_path2)

Window length 200; Predicting test signal 0 out of 5
Window length 200; Predicting test signal 1 out of 5
Window length 200; Predicting test signal 2 out of 5
Window length 200; Predicting test signal 3 out of 5
Window length 200; Predicting test signal 4 out of 5
